# Recurrent Neural Networks Tutorial

In HW1 you've worked on building simple MLP models that you used to classify single inputs of MNIST images into ten classes, and in HW2 you will work with CNNs to recognize objects in images. However, real world data are rarely standalone fixed-size vectors that do not depend on context - speech segments, for instance, have arbitrary lengths, and evidently depend on context. In this tutorial, we will take a small step in this direction by building a basic RNN for generating text by learning from some sample text files that we provide.

# You must random walk before you rnn

Let's begin by loading a choice selection of tweets by the leader of the free world, courtesy a dataset available on Kaggle (https://www.kaggle.com/kingburrito666/better-donald-trump-tweets), formatted for this recitation*.

\**11785 staff are not responsible for content, accuracy, or sanity of this dataset*

In [1]:
import numpy as np
from collections import Counter

history_length = 3

def process_input(filename):
    rawtext = open(filename).read()
    sequences = [rawtext[i:i+history_length] for i in range(int(len(rawtext)/history_length))]
    stats = Counter(sequences)
    tokens = []
    counts = []
    for i in stats.most_common():
        tokens.append(i[0])
        counts.append(i[1])
    return stats
    
def next_char(cur,stats):
    seed = cur[1:]
    candidates = []
    candidatec = []
    for k in stats.keys():
        if seed==k[:-1]:
            candidates.append(k)
            candidatec.append(float(stats[k]))
    candidatep = [x/sum(candidatec) for x in candidatec]
    return candidates[np.random.choice(len(candidatec),p=candidatep)]

def sample(length,running_state, stats):
    output = ''
    for i in range(length):
        output+=running_state[0]
        running_state = next_char(running_state,stats)
    return output

In [2]:
tweet_stats = process_input('tweets.txt')
sample(50,'oba',tweet_stats)

'oban an t co bad firs  the  speco wor busins han h'

To convince you that this is a reasonable approach, let's take a more reasonable example of the English language:

In [3]:
hamlet_stats = process_input('hamlet.txt')
sample(50, 'the',hamlet_stats)

"the min to mings fore's to sand be: the them? thei"

And if we further increase the running history that this extremely simple model keeps, we see better and better examples:

In [5]:
history_length = 4
tweet_stats = process_input('tweets.txt')
print(sample(50,'and ',tweet_stats))
hamlet_stats = process_input('hamlet.txt')
print(sample(50,'the ',hamlet_stats))

and  vegallowers   new https     your see   t come
the that fles, and by a sea of outrageousand them?


In [6]:
history_length = 5
tweet_stats = process_input('tweets.txt')
print(sample(50,'obama',tweet_stats))
hamlet_stats = process_input('hamlet.txt')
print(sample(50,'to be',hamlet_stats))

obamas belief effortunited past the field  carolin
to be, or not to say we end the question: whether 


However, there are some inherent limitations to this approach. Character-level n-grams do get better with increasing n, but we run into all sorts of limitations in terms of memory (possible combinations are exponential in the size of alphabet) and sparsity (it is HIGHLY unlikely that you will observe all possible n-grams in the data). There exists a lot of literature over the past few decades dealing with engineering NLP techniques to get around this (cf. [1]), but even then we are limited by the fact that information from (n+1) steps ago gets washed out.


[1] Chen and Goodman. An Empirical Study of Smoothing Techniques for Language Modeling. 1998.

# Recurrent Neural Networks

So far, we've been working with networks where the input size was fixed. Today let's take the example of English language. A typical method of vectorizing words is to convert them into a one-hot representation over the vocabulary.

Let's take one such sample vector representation x which belongs to the set of all such vectors X (the vocabulary).

In the previous section, we implemented a Markov chain that generates sentences given a seed and statistics over the training data. From a high-level standpoint, we were looking at the following method:

In [7]:
# This code is meant as an overview, not meant to compile
'''
def markov_generator(x):
    current_state = current_state
    for i in range(some target length):
        output += generate_single_step_output(current_state)
        new_state = random_process(current_state)
        current_state = new_state
    return output
'''

'\ndef markov_generator(x):\n    current_state = current_state\n    for i in range(some target length):\n        output += generate_single_step_output(current_state)\n        new_state = random_process(current_state)\n        current_state = new_state\n    return output\n'

Here, a string of length n corresponds to the current state. Evidently, forgetting your previous state at every step is not going to be able to capture long term dependencies. We have to stop being Markovian!

Instead, what we need is a method of *storing* what we have observed so far. Consider the following method:

In [8]:
# This code is meant as an overview, not meant to compile
'''
class generator_with_memory:
    def __init__():
        self.memory_state = self.init_memory()
    def step(x):
        self.memory_state = smart_process(x,self.memory_state)
        y = generate_output(self.memory_state)
        return y
'''

'\nclass generator_with_memory:\n    def __init__():\n        self.memory_state = self.init_memory()\n    def step(x):\n        self.memory_state = smart_process(x,self.memory_state)\n        y = generate_output(self.memory_state)\n        return y\n'

By defining an object with an associated memory_state and a smart choice of smart_process() and generate_output(), we might be able to capture long term dependencies by *storing* them in our memory_state.

How can we create such functions? One choice could be to have a linear transform between all associated vectors. Let's look at our vectors so far. Let's call memory_state as h for convenience.

x: Input vector

y: Output vector

h: "Hidden" memory_state vector

In [9]:
# This code is meant as an overview, not meant to compile
'''
def step(x):
    self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
    y = np.dot(self.W_hy, self.h)
    return y
'''

'\ndef step(x):\n    self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))\n    y = np.dot(self.W_hy, self.h)\n    return y\n'

The update step incorporates both the current input x and the current memory state h. This can replace smart_process above and incorporate memory into our model! We can initialize the model parameters W_hh, W_xh, W_hy randomly and learn them from our data.

It is also important to note that this method is able to work with arbitrary length sequences. We can just step through the entire sequence and rely on h to keep track of long range dependencies. Thus we have moved away from the fixed input size paradigm as well! You can visualize the process of stepping through the sequence from the following image (credits: Wikipedia)

<img src="rnn.svg" width=700>

You can come up with more imaginative ways to step through the sequence. For instance, why should we restrict ourselves to a flat sequence? A step combines the current input x_t with the current hidden state h_t. One can recursively apply this combination step, such as the authors of [2] who used this idea for sentiment analysis. 


<img src="recursive1.png" width=400>

One can also stack things - instead of having just one recurrent layer you can use

y1 = rnn1.step(x)

y2 = rnn2.step(y1)

and so on.

[2] Socher, Manning, Ng. Recursive deep models for semantic compositionality over a sentiment treebank. EMNLP 2013.


# RNNs using PyTorch

Let's work with an example task and implement an RNN using PyTorch. Specifically, let's look at a simple time series prediction problem. For the purpose of this recitation we will work on a very simple time series prediction problem adapted from PyTorch examples [3]. We'll generate a random set of sinusoidal waves and use LSTMCell units to learn these waves and predict the next few steps.


[3] https://github.com/pytorch/examples/tree/master/time_sequence_prediction

In [10]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

np.random.seed(0)
torch.manual_seed(0)

T = 80    # range from which ints are sampled
L = 1000  # Length of generated sequence
N = 100   # number of examples
future = 1000 # length of sequence to predict
# generating a sinusoidal time series
x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(- 4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')


class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 32)
        self.lstm2 = nn.LSTMCell(32, 32)
        self.linear = nn.Linear(32, 1)
    def forward(self, input, future = 0):
        outputs = []
        h_t = Variable(torch.zeros(input.size(0), 32).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), 32).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), 32).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), 32).double(), requires_grad=False)
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# predicting future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs
    
def save_plot_wave(y_gen):
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequence', fontsize=30)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.plot(np.arange(input.size(1)), y_gen[0][:input.size(1)], 'b', linewidth = 2.0)
    plt.plot(np.arange(input.size(1), input.size(1) + future), y_gen[0][input.size(1):], 'b' + ':', linewidth = 2.0)
    plt.savefig('predict%d.pdf'%i)
    plt.close()

Notice the staggered nature of input and target data below:

In [11]:
input = Variable(torch.from_numpy(data[1:, :-1]), requires_grad=False)
target = Variable(torch.from_numpy(data[1:, 1:]), requires_grad=False)
test_input = Variable(torch.from_numpy(data[:1, :-1]), requires_grad=False)
test_target = Variable(torch.from_numpy(data[:1, 1:]), requires_grad=False)

Here we will use the LBFGS optimizer. It is used for parameter estimation by minimizing a smooth f(x) over unconstrained real-valued vector x. It needs to reevaluate the function multiple times, so you have to pass in a closure that allows recomputation. The closure should clear the gradients, compute the loss, and return it [4]. LBFGS also does not support per-parameter options.


[4] http://pytorch.org/docs/master/optim.html#optimizer-step-closure

In [ ]:
# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
#begin to train
for i in range(11):
    print('Step: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('Loss:', loss.data.numpy()[0])
        loss.backward()
        return loss
    optimizer.step(closure)
    # begin to predict
    pred = seq(test_input, future = future)
    loss = criterion(pred[:, :-future], test_target)
    print('Test loss:', loss.data.numpy()[0])
    y = pred.data.numpy()
    save_plot_wave(y)

Step:  0
Loss: 0.534735817744
Loss: 0.51595643509
Loss: 0.487655250214
Loss: 0.476866620552
Loss: 0.443208983365
Loss: 0.354411918037
Loss: 0.230974896977
Loss: 6.21625536659
Loss: 0.107385785838
Loss: 0.0244810777848
Loss: 0.00949078034371
Loss: 0.00899265775527
Loss: 0.00716001025314
Loss: 0.00574255749538
Loss: 0.00422940005235
Loss: 0.00372545641453
Loss: 0.00351897569446
Loss: 0.00316297494398
Loss: 0.00197494376456
Loss: 0.0012997484383
Test loss: 0.00116922468676
Step:  1
Loss: 0.00145462772409
Loss: 0.00118182828882
Loss: 0.00114993069323
Loss: 0.00111182542918
Loss: 0.00104719865128
Loss: 0.000952630861115
Loss: 0.000879430665862
Loss: 0.000858234694016
Loss: 0.000823140827356
Loss: 0.000815607769945
Loss: 0.000785582041389
Loss: 0.000768247239385
Loss: 0.000739432594294
Loss: 0.000691610451779
Loss: 0.000639641483622
Loss: 0.00056794222483
Loss: 0.000582874034494
Loss: 0.000521702656905
Loss: 0.000527696125628
Loss: 0.000490990391194
Test loss: 0.000364178401968
Step:  2
Loss